In [1]:
import streamlit as st

with st.echo():
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager

    @st.experimental_singleton
    def get_driver():
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--headless')

    driver = get_driver()
    driver.get("http://example.com")

    st.code(driver.page_source)

In [5]:
from selenium import webdriver 
import time
driver = webdriver.PhantomJS(executable_path='<PhantomJS Path Here>') 
driver.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html') 
time.sleep(3)
print(driver.find_element_by_id('content').text)
driver.close()

SyntaxError: invalid syntax (2966543922.py, line 1)

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.
